In [2]:
import pypyodbc 
import pandas as pd

cnxn = pypyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=.,1433;"
                        "Database=btcSmartTrader;"
                        "uid=sa;pwd=P@ssw0rd")
df = pd.read_sql_query('select * from dbo.bitcoinTwitter', cnxn, parse_dates=['timestamp'])

In [6]:
df['polarity'] = 0

In [7]:
print(df.columns.values)

['id' 'priceopen' 'priceclose' 'pricechange' 'timestamp' 'tweetid'
 'tweetcreatedate' 'tweetlang' 'tweettext' 'polarity']


In [8]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))

    return analysis.sentiment.polarity

df['polarity'] = df.apply(lambda row: get_tweet_sentiment(row['tweettext']), axis=1)


In [9]:
print(df[:3])

   id  priceopen  priceclose  pricechange           timestamp  \
0   1   11230.96    11270.25     0.003498 2018-01-25 20:21:42   
1   2   11230.96    11270.25     0.003498 2018-01-25 20:21:42   
2   3   11230.96    11270.25     0.003498 2018-01-25 20:21:42   

              tweetid     tweetcreatedate tweetlang  \
0  956623229958983681 2018-01-25 20:22:05        en   
1  956623227450810368 2018-01-25 20:22:05        en   
2  956623229061402624 2018-01-25 20:22:05        en   

                                           tweettext  polarity  
0  RT @netflix: FRIENDS 2018 PLOTLINES-Joey eats ...       0.0  
1  You Can Now Use Bitcoin as Collateral When App...       0.0  
2  RT @KenanMusaev: Progress of development... #L...       0.0  


In [10]:
df.to_csv("bitcoinTwitterPolarity.csv")